In [31]:
import itertools
import random
import numpy

ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
suits = ['♣', '♦', '♥', '♠']
deck = ['{}{}'.format(i[0],i[1]) for i in list(itertools.product(ranks, suits))]

ranks_weight = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7,  '8':8,  '9':9,  'T':10,  'J':11,  'Q':12,  'K':13, 'A':14}
suits_weight = {'♣':1, '♦':2, '♥':3, '♠':4}

def get_sorted_rank(H):
    return sorted(H, key=lambda k: ranks_weight[k[0]], reverse=True)

def get_sorted_suit(H):
    return sorted(H, key=lambda k: suits_weight[k[1]], reverse=True)

def highest_rank(H):
    return H[:1], H[1:]

def highest_pair(H):
    r = [],H
    for i in range(len(H)-1):
        if H[i][0]==H[i+1][0]:
            r = H[i:i+2],H[0:i]+H[i+2:]
            break
    return r

def highest_2_pairs(H):
    r = [],H
    a,b = highest_pair(H[:])
    c,d = highest_pair(b[:]) if a else (False, None)
    if c: r = a+c,d
    return r

def highest_three(H):
    r = [],H
    for i in range(len(H)-2):
        if H[i][0]==H[i+1][0]==H[i+2][0]:
            r = H[i:i+3],H[0:i]+H[i+3:]
            break
    return r

def highest_straight(H):
    r = [],[i for i in H]
    v0 = 6 if H[0][0]=='A' else 0
    U,D = [],[]
    last = H.pop(0)
    U.append(last)
    for i in range(len(H)):
        new = H.pop(0)
        if last[0]==new[0]:
            D.append(new)
        else:
            U.append(new)
            last = new
    v1=v2=v3=v4=v5=-1
    for i in range(len(U)-4):
        v1 = ranks_weight[U[i][0]]
        v2 = ranks_weight[U[i+1][0]]+1
        v3 = ranks_weight[U[i+2][0]]+2
        v4 = ranks_weight[U[i+3][0]]+3
        v5 = ranks_weight[U[i+4][0]]+4
        if v1==v2==v3==v4==v5:
            r = U[i:i+5],get_sorted_rank(U[i+5:]+U[0:i]+D)
            break
    if v0==v2==v3==v4==v5:
        r = U[-4:]+U[0:1],get_sorted_rank(D+U[1:-4])

    return r

def highest_flush(H):
    H = get_sorted_suit(H[:])
    r = [],H
    for i in range(len(H)-4):
        if H[i][1]==H[i+1][1]==H[i+2][1]==H[i+3][1]==H[i+4][1]:
            r = get_sorted_rank(H[i:i+5]), get_sorted_rank(H[0:i]+H[i+5:])
            break
    return r

def highest_full_house(H):
    r = [],H
    a,b = highest_three(H[:])
    c,d = highest_pair(b[:]) if a else (False, None)
    if c: r = a+c,d
    return r

def highest_four(H):
    r = [],H
    for i in range(len(H)-3):
        if H[i][0]==H[i+1][0]==H[i+2][0]==H[i+3][0]:
            r = H[i:i+4],H[0:i]+H[i+4:]
            break
    return r


def _highest_count_suit(H):
    S = {}
    for k,v in suits_weight.items():
        S[v] = []
    for i in range(len(H)):
        new = H.pop(0)
        S[suits_weight[new[1]]].append(new)
    m = -1
    l = -1
    for k,v in suits_weight.items():
        x = len(S[v])
        if x>m: 
            m=x
            l=v
    r1,r2 =[],[]
    for k,v in suits_weight.items():
        if v==l:
            r1 = r1 + S[v]
        else:
            r2 = r2 + S[v]
    return get_sorted_rank(r1),get_sorted_rank(r2)

def highest_straight_flush(H):
    r = [],H
    a,b = _highest_count_suit(H[:])
    c,d = highest_straight(a[:]) if len(a)>=5 else (False, None)
    if c: r = c,get_sorted_rank(b+d)
    return r

def highest_royal_straight_flush(H):
    r = [],H
    a,b = highest_straight_flush(H[:])
    if a and a[0][0]=='A': r = a,b
    return r

hand_names = {
    1: 'highest_rank',
    2: 'highest_pair',
    3: 'highest_2_pairs',
    4: 'highest_three',
    5: 'highest_straight',
    6: 'highest_flush',
    7: 'highest_full_house',
    8: 'highest_four',
    9: 'highest_straight_flush',
    10:'highest_royal_straight_flush'
}

def check_hand(t, h=[], check=False):
    H = h+t
    hand = get_sorted_rank(H)
    ranking = {
        10:highest_royal_straight_flush,
        9: highest_straight_flush,
        8: highest_four,
        7: highest_full_house,
        6: highest_flush,
        5: highest_straight,
        4: highest_three,
        3: highest_2_pairs,
        2: highest_pair,
        1: highest_rank
    }
    
    rank, best_hand = 0, []
    
    for i in range(10,0,-1):
        h,r = ranking[i](hand[:])
        if h:
            rank, best_hand = i, h
            break
            
    tie = [ranks_weight[u[0]] for u in best_hand]
 
    return [rank]+tie, best_hand, check

def check_table(table, hand_list):
    results = []
    for i in range(len(hand_list)):
        hand = hand_list[i]
        results.append(check_hand(table, hand, i==0))
        
    results = sorted(results, key=lambda k: k[0]+[k[2]], reverse=True)
    won = results[0][2]
    tie = results[0][0]==results[1][0]
    
    return won, tie, results

def check_random(d, N=2):
    H=2
    T=5
    sample = random.sample(d, T + H*N)
    game = sample[:T],[sample[T+i*H:T+(i+1)*H] for i in range(N)]
    won, tie, result = check_table(game[0], game[1])
    result = sorted(result, key=lambda k: k[0]+[k[2]], reverse=True)
    return won, tie, result, game

def check_hand_random(d, h, t=[], N_PLAYERS=2):
    N=N_PLAYERS-1
    H=2
    T=5-len(t)
    deck_copy = d.copy()
    for card in h+t:
        deck_copy.remove(card)
    sample = random.sample(deck_copy, T+H*N)
    t = t + sample[:T]
    opponent_hands = [sample[T+i*H:T+(i+1)*H] for i in range(N)]
    won, tie, result = check_table(t, [h]+opponent_hands)
    return won, tie, result, [t, [h]+opponent_hands]

def check_hand_random_n(d, h, t=[], n=100, N_PLAYERS=2):
    N = N_PLAYERS-1
    H=2
    T=5-len(t)
    deck_copy = d.copy()
    for card in h+t:
        deck_copy.remove(card)
    dict_turns = {'won':0.0, 'tie':0.0, 'lost':0.0}
    step = 1/n
    for i in range(n):
        sample = random.sample(deck_copy, T+H*N)
        tf = t + sample[:T]
        opponent_hands = [sample[T+i*H:T+(i+1)*H] for i in range(N)]
        won, tie, result = check_table(tf, [h]+opponent_hands)
        if not won:
            dict_turns['lost'] += step
        elif won and not tie:
            dict_turns['won'] += step
        elif won and tie:
            dict_turns['tie'] += step
            
    for key in dict_turns.keys():
        dict_turns[key] = round(dict_turns[key], 3)
        
    return dict_turns

def gen_game(N=2, H=2, T=5):
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
    suits = ['♣', '♦', '♥', '♠']
    deck = ['{}{}'.format(i[0],i[1]) for i in list(itertools.product(ranks, suits))]
    sample = random.sample(deck, T + H*N)
    return sample[:T],[sample[T+i*H:T+(i+1)*H] for i in range(N)],deck

def run_turn(N=2, H=2, T=5, show=3, n=10000):
    t,hands,deck = gen_game(N, H, T)
    print(get_sorted_rank(t[:show]),hands[0])
    r = float(input('prob: '))
    s = check_table(t, hands)
    state = 'lost'
    if s[0] and not s[1]:
        state='won'
    elif s[0] and s[1]:
        state='tie'
    print('state: ', state)
    print('table: ', get_sorted_rank(t))
    for i in range(len(s[2])):
        print('P{}{}: '.format(i,'*' if s[2][i][2] else ''), s[2][i][1])

    p = check_hand_random_n(deck, hands[0],t[:show], n, N)
    print(p)
    print('expected: ', round(N*p['won']+0.5*N*p['tie']-p['lost'], 3))
    print()
    return r-p['won']-0.5*p['tie']

In [32]:
l = []
for i in range(100):
    l.append(run_turn(2))
    x = input('finish: ')
    if x: break
round(numpy.mean(l),2),round(numpy.std(l),2)

['T♥', '7♥', '3♣'] ['2♦', '4♦']
prob: 1
state:  lost
table:  ['T♥', '9♦', '7♥', '3♣', '2♠']
P0:  ['9♣', '9♦']
P1*:  ['2♦', '2♠']
{'tie': 0.256, 'won': 0.137, 'lost': 0.607}
expected:  -0.077

finish: 1


(0.73999999999999999, 0.0)

In [33]:
txt = """
1suit 45 %

2samesuit 6 %

1rank - 4.2% each in deck
    1specific 4.2 %

2rank - 0.37% each in deck
    2specific 0.1 %
    
gambler's ruin'

if p=0.5 => $p = n1/(n1+n2)$

"""

In [69]:
class b(Poker):
    def num_players(self):
        return 2
    def max_hand(self):
        return 2
    def max_table(self):
        return 5
x = b()
x.shuffle(players=False)

[False, False] True
['5♣', '3♠', '8♠', '9♥', 'A♥', '3♥', '2♥', 'J♣', '7♦', 'J♥', '6♦', '6♥', 'J♦', 'T♠', '5♥', 'K♣', 'T♣', '5♠', 'A♣', '7♥', '7♣', '4♦', '2♦', '8♣', '8♦', '2♣', '6♣', '8♥', '4♠', 'T♦', '4♥', '7♠', '4♣', 'K♠', 'Q♠', 'Q♦', 'A♦', 'Q♣', '9♦', 'K♦', '2♠', '6♠', 'K♥']
[['9♣', '3♣'], ['9♠', 'J♠']] ['3♦', 'A♠', 'Q♥', '5♦', 'T♥'] ['K♦', '6♥', '2♥', '5♥', '7♦', 'J♥', 'Q♣', '2♦', 'T♦', 'J♣', '5♦', '8♣', '8♥', '7♥', '9♥', 'J♦', 'K♣', '6♣', 'A♠', 'Q♦', '4♠', 'T♠', '5♠', '8♠', '3♠', '3♦', 'A♣', '7♣', '9♦', '6♦', 'A♦', '2♣', '4♣', 'Q♠', 'K♥', '6♠', '4♦', 'T♣', '7♠', '4♥', 'Q♥', 'K♠', 'T♥', 'A♥', '8♦', '5♣', '2♠', '3♥']


In [67]:
"""
class Hand -> get sorted, highest, hand names

class game -> new, random, turns, etc
"""

class _classproperty(property):
    def __get__(self, cls, owner):
        return classmethod(self.fget).__get__(None,owner)()
    
class _staticproperty(property):
    def __get__(self, cls, owner):
        return staticmethod(self.fget).__get__(owner)()

from abc import ABC, abstractmethod
    
class Poker(ABC):
    
    @_staticproperty
    def ranks():
        return ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
    
    @_staticproperty
    def suits():
        return ['♣', '♦', '♥', '♠']
    
    @_classproperty
    def deck(cls):
        return ['{}{}'.format(i[0],i[1]) for i in list(itertools.product(cls.ranks, cls.suits))]
    
    @abstractmethod
    def num_players(self):
        pass
    
    @abstractmethod
    def max_hand(self):
        pass
    
    @abstractmethod
    def max_table(self):
        pass
    
    def __init__(self):
        self.new_game()
    
    def new_game(self, my_deck=False):
        import random
        s = my_deck if my_deck else random.sample(self.deck, len(self.deck))
        p = self.num_players()
        h = self.max_hand()
        t = self.max_table()
        self._table =  [s.pop(0) for _ in range(t)]
        self._players = [[s.pop(0) for _ in range(h)] for _ in range(p)]
        self._deck = s
    
    def shuffle(self, players=True, deck=True):
        p = self.num_players()
        h = self.max_hand()
        t = self.max_table()
        try:
            players = [bool(players[i]) for i in range(p)]
        except:
            b = True if players is True else False
            players = [b for i in range(p)]
        print(players, deck)
        to_shuffle = self._deck
        if deck: to_shuffle = to_shuffle + self._table
        for i in range(len(players)):
            if players[i]: to_shuffle = to_shuffle + self._players[i]
        random.shuffle(to_shuffle)
        print(self._deck)
        print(self._players, self._table, to_shuffle)
        
        #my_deck =[s.pop(0) for _ in range(t)]

    def set_hand(self, i, hand):
        pass
    
    def set_table(self, table):
        pass
    
    @classmethod
    def compare_hands(cls, hand_a, hand_b):
        a = cls.evaluate_hand(hand_a)
        b = cls.evaluate_hand(hand_b)
        return hand_a if a>b else hand_b
    
    def evaluate_hand(cls, hand):
        ranks_weight = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7,  '8':8,  
                        '9':9,  'T':10,  'J':11,  'Q':12,  'K':13, 'A':14}
        suits_weight = {'♣':1, '♦':2, '♥':3, '♠':4}
        return ranks_weight[hand[0]]*suits_weight[hand[1]]
    
    def winner(self):
        pass

    
    
class TexasHoldem(Poker):

    def __init__(self, hand, table=None):
        pass

import itertools
import random
import numpy

ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
suits = ['♣', '♦', '♥', '♠']
deck = ['{}{}'.format(i[0],i[1]) for i in list(itertools.product(ranks, suits))]

ranks_weight = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7,  '8':8,  '9':9,  'T':10,  'J':11,  'Q':12,  'K':13, 'A':14}
suits_weight = {'♣':1, '♦':2, '♥':3, '♠':4}

def get_sorted_rank(H):
    return sorted(H, key=lambda k: ranks_weight[k[0]], reverse=True)

def get_sorted_suit(H):
    return sorted(H, key=lambda k: suits_weight[k[1]], reverse=True)

def highest_rank(H):
    return H[:1], H[1:]

def highest_pair(H):
    r = [],H
    for i in range(len(H)-1):
        if H[i][0]==H[i+1][0]:
            r = H[i:i+2],H[0:i]+H[i+2:]
            break
    return r

def highest_2_pairs(H):
    r = [],H
    a,b = highest_pair(H[:])
    c,d = highest_pair(b[:]) if a else (False, None)
    if c: r = a+c,d
    return r

def highest_three(H):
    r = [],H
    for i in range(len(H)-2):
        if H[i][0]==H[i+1][0]==H[i+2][0]:
            r = H[i:i+3],H[0:i]+H[i+3:]
            break
    return r

def highest_straight(H):
    r = [],[i for i in H]
    v0 = 6 if H[0][0]=='A' else 0
    U,D = [],[]
    last = H.pop(0)
    U.append(last)
    for i in range(len(H)):
        new = H.pop(0)
        if last[0]==new[0]:
            D.append(new)
        else:
            U.append(new)
            last = new
    v1=v2=v3=v4=v5=-1
    for i in range(len(U)-4):
        v1 = ranks_weight[U[i][0]]
        v2 = ranks_weight[U[i+1][0]]+1
        v3 = ranks_weight[U[i+2][0]]+2
        v4 = ranks_weight[U[i+3][0]]+3
        v5 = ranks_weight[U[i+4][0]]+4
        if v1==v2==v3==v4==v5:
            r = U[i:i+5],get_sorted_rank(U[i+5:]+U[0:i]+D)
            break
    if v0==v2==v3==v4==v5:
        r = U[-4:]+U[0:1],get_sorted_rank(D+U[1:-4])

    return r

def highest_flush(H):
    H = get_sorted_suit(H[:])
    r = [],H
    for i in range(len(H)-4):
        if H[i][1]==H[i+1][1]==H[i+2][1]==H[i+3][1]==H[i+4][1]:
            r = get_sorted_rank(H[i:i+5]), get_sorted_rank(H[0:i]+H[i+5:])
            break
    return r

def highest_full_house(H):
    r = [],H
    a,b = highest_three(H[:])
    c,d = highest_pair(b[:]) if a else (False, None)
    if c: r = a+c,d
    return r

def highest_four(H):
    r = [],H
    for i in range(len(H)-3):
        if H[i][0]==H[i+1][0]==H[i+2][0]==H[i+3][0]:
            r = H[i:i+4],H[0:i]+H[i+4:]
            break
    return r


def _highest_count_suit(H):
    S = {}
    for k,v in suits_weight.items():
        S[v] = []
    for i in range(len(H)):
        new = H.pop(0)
        S[suits_weight[new[1]]].append(new)
    m = -1
    l = -1
    for k,v in suits_weight.items():
        x = len(S[v])
        if x>m: 
            m=x
            l=v
    r1,r2 =[],[]
    for k,v in suits_weight.items():
        if v==l:
            r1 = r1 + S[v]
        else:
            r2 = r2 + S[v]
    return get_sorted_rank(r1),get_sorted_rank(r2)

def highest_straight_flush(H):
    r = [],H
    a,b = _highest_count_suit(H[:])
    c,d = highest_straight(a[:]) if len(a)>=5 else (False, None)
    if c: r = c,get_sorted_rank(b+d)
    return r

def highest_royal_straight_flush(H):
    r = [],H
    a,b = highest_straight_flush(H[:])
    if a and a[0][0]=='A': r = a,b
    return r

hand_names = {
    1: 'highest_rank',
    2: 'highest_pair',
    3: 'highest_2_pairs',
    4: 'highest_three',
    5: 'highest_straight',
    6: 'highest_flush',
    7: 'highest_full_house',
    8: 'highest_four',
    9: 'highest_straight_flush',
    10:'highest_royal_straight_flush'
}

def check_hand(t, h=[], check=False):
    H = h+t
    hand = get_sorted_rank(H)
    ranking = {
        10:highest_royal_straight_flush,
        9: highest_straight_flush,
        8: highest_four,
        7: highest_full_house,
        6: highest_flush,
        5: highest_straight,
        4: highest_three,
        3: highest_2_pairs,
        2: highest_pair,
        1: highest_rank
    }
    
    rank, best_hand = 0, []
    
    for i in range(10,0,-1):
        h,r = ranking[i](hand[:])
        if h:
            rank, best_hand = i, h
            break
            
    tie = [ranks_weight[u[0]] for u in best_hand]
 
    return [rank]+tie, best_hand, check

def check_table(table, hand_list):
    results = []
    for i in range(len(hand_list)):
        hand = hand_list[i]
        results.append(check_hand(table, hand, i==0))
        
    results = sorted(results, key=lambda k: k[0]+[k[2]], reverse=True)
    won = results[0][2]
    tie = results[0][0]==results[1][0]
    
    return won, tie, results

def check_random(d, N=2):
    H=2
    T=5
    sample = random.sample(d, T + H*N)
    game = sample[:T],[sample[T+i*H:T+(i+1)*H] for i in range(N)]
    won, tie, result = check_table(game[0], game[1])
    result = sorted(result, key=lambda k: k[0]+[k[2]], reverse=True)
    return won, tie, result, game

def check_hand_random(d, h, t=[], N_PLAYERS=2):
    N=N_PLAYERS-1
    H=2
    T=5-len(t)
    deck_copy = d.copy()
    for card in h+t:
        deck_copy.remove(card)
    sample = random.sample(deck_copy, T+H*N)
    t = t + sample[:T]
    opponent_hands = [sample[T+i*H:T+(i+1)*H] for i in range(N)]
    won, tie, result = check_table(t, [h]+opponent_hands)
    return won, tie, result, [t, [h]+opponent_hands]

def check_hand_random_n(d, h, t=[], n=100, N_PLAYERS=2):
    N = N_PLAYERS-1
    H=2
    T=5-len(t)
    deck_copy = d.copy()
    for card in h+t:
        deck_copy.remove(card)
    dict_turns = {'won':0.0, 'tie':0.0, 'lost':0.0}
    step = 1/n
    for i in range(n):
        sample = random.sample(deck_copy, T+H*N)
        tf = t + sample[:T]
        opponent_hands = [sample[T+i*H:T+(i+1)*H] for i in range(N)]
        won, tie, result = check_table(tf, [h]+opponent_hands)
        if not won:
            dict_turns['lost'] += step
        elif won and not tie:
            dict_turns['won'] += step
        elif won and tie:
            dict_turns['tie'] += step
            
    for key in dict_turns.keys():
        dict_turns[key] = round(dict_turns[key], 3)
        
    return dict_turns

def gen_game(N=2, H=2, T=5):
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
    suits = ['♣', '♦', '♥', '♠']
    deck = ['{}{}'.format(i[0],i[1]) for i in list(itertools.product(ranks, suits))]
    sample = random.sample(deck, T + H*N)
    return sample[:T],[sample[T+i*H:T+(i+1)*H] for i in range(N)],deck

def run_turn(N=2, H=2, T=5, show=3, n=10000):
    t,hands,deck = gen_game(N, H, T)
    print(get_sorted_rank(t[:show]),hands[0])
    r = float(input('prob: '))
    s = check_table(t, hands)
    state = 'lost'
    if s[0] and not s[1]:
        state='won'
    elif s[0] and s[1]:
        state='tie'
    print('state: ', state)
    print('table: ', get_sorted_rank(t))
    for i in range(len(s[2])):
        print('P{}{}: '.format(i,'*' if s[2][i][2] else ''), s[2][i][1])

    p = check_hand_random_n(deck, hands[0],t[:show], n, N)
    print(p)
    print('expected: ', round(N*p['won']+0.5*N*p['tie']-p['lost'], 3))
    print()
    return r-p['won']-0.5*p['tie']

In [77]:
class classproperty(property):
    def __get__(self, cls, owner):
        return classmethod(self.fget).__get__(None, owner)()

class Poker:
    
    @staticmethod
    @property
    def rank():
        return ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
    
x = Poker()

Poker.rank